<a href="https://colab.research.google.com/github/sivasamy1/Machine_learning_research/blob/main/Stock_price_prediction_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [9]:
def prepare_data(df,forecast_col,forecast_out,test_size):
    label = df[forecast_col].shift(-forecast_out) #creating new column called label with the last 5 rows are nan
    X = np.array(df[[forecast_col]]) #creating the feature array
    X = preprocessing.scale(X) #processing the feature array
    X_lately = X[-forecast_out:] #creating the column i want to use later in the predicting method
    X = X[:-forecast_out] # X that will contain the training and testing
    label.dropna(inplace=True) #dropping na values
    y = np.array(label)  # assigning Y
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_size, random_state=0) #cross validation

    response = [X_train,X_test , Y_train, Y_test , X_lately]
    return response

In [36]:
df = pd.read_csv("Bitcoin_8_30_2024-8_30_2025_historical_data_coinmarketcap.csv", delimiter=';')
# df = df[df.name == "2781"]

In [37]:
print(tabulate(df, headers='keys', tablefmt='github'))

|     | timeOpen                 | timeClose                | timeHigh                 | timeLow                  |   name |     open |     high |      low |    close |      volume |   marketCap | timestamp                |
|-----|--------------------------|--------------------------|--------------------------|--------------------------|--------|----------|----------|----------|----------|-------------|-------------|--------------------------|
|   0 | 2025-08-29T00:00:00.000Z | 2025-08-29T23:59:59.999Z | 2025-08-29T00:30:00.000Z | 2025-08-29T21:04:00.000Z |   2781 | 112551   | 112619   | 107560   | 108411   | 7.78434e+10 | 2.15881e+12 | 2025-08-29T23:59:59.999Z |
|   1 | 2025-08-28T00:00:00.000Z | 2025-08-28T23:59:59.999Z | 2025-08-28T13:23:00.000Z | 2025-08-28T00:36:00.000Z |   2781 | 111219   | 113450   | 110901   | 112545   | 5.88602e+10 | 2.24121e+12 | 2025-08-28T23:59:59.999Z |
|   2 | 2025-08-27T00:00:00.000Z | 2025-08-27T23:59:59.999Z | 2025-08-27T14:36:00.000Z | 2025-08-27T08:5

In [113]:
forecast_col = 'close'
forecast_out = 3
test_size = 0.774

In [114]:
X_train, X_test, Y_train, Y_test , X_lately =prepare_data(df,forecast_col,forecast_out,test_size); #calling the method where the cross validation and data preperation is in
learner = LinearRegression() #initializing linear regression model

learner.fit(X_train,Y_train) #training the linear regression model

LinearRegression()

In [115]:
score=learner.score(X_test,Y_test)#testing the linear regression model
forecast= learner.predict(X_lately) #set that will contain the forecasted data
response={}#creting json object
response['test_score']=score
response['forecast_set']=forecast

print(response)

{'test_score': 0.9629625041515603, 'forecast_set': array([59595.0082531 , 57853.74171018, 59456.075458  ])}
